In [2]:
import pandas as pd

from pathlib import Path

/Users/jonathanlim/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
Path.cwd()

PosixPath('/Users/jonathanlim/workspace/mila/winter_2023/IFT6759_Advanced_Projects/project/quick-recipe/notebooks')

In [9]:
PROJ_DIR = Path.cwd().parent
DATA_DIR = PROJ_DIR / 'data'
LOG_DIR = PROJ_DIR / 'logs'

LOG_DIR.mkdir(parents=True, exist_ok=True) 
LOG_SAVE_DIR = LOG_DIR / 'youcook_attnlstm_multimodal'

In [17]:
master_df = pd.read_pickle(DATA_DIR / 'full_master_updated.pkl')
val_df = pd.read_csv(str(LOG_SAVE_DIR / 'val_performance.csv'))

## Generate Knowledge Extraction metrics based on best Key-clip Prediction model's predictions

In [15]:
def best_keyclip_prediction_true_positives(row):
    return row['true_positives'] if row['IsPredUseful'] == 1 else 0

def generate_metrics_from_master_and_val_df(master_df, val_df, master_key='Sentence', val_key='text'):
  combined_df = pd.merge(master_df, val_df, left_on=master_key, right_on=val_key, how='inner')
  combined_df['best_kc_pred_true_positives'] = combined_df.apply(lambda row: best_keyclip_prediction_true_positives(row), axis=1)
  total_num_gold = combined_df['num_gold'].sum()
  total_num_predicted = combined_df['num_predicted'].sum()
  total_true_pos = combined_df['best_kc_pred_true_positives'].sum()
  precision = round(total_true_pos / total_num_predicted, 2)
  recall = round(total_true_pos / total_num_gold, 2)
  f1 = round(2*(precision*recall) / (precision+recall), 2)

  print(f"Total true positives (predicted words overlap with gold words): {total_true_pos}")
  print(f"Total predicted words: {total_num_predicted}")
  print(f"Total gold words: {total_num_gold}")
  print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
  
  return precision, recall, f1

In [22]:
master_df

,VideoId,Split,Sentence,IsUsefulSentence,Key steps,Verb,Obj,img_feat,text_feat(all-MiniLM-L6-v2),VideoUrl,text_encodings(Attn-LSTM),text_encodings_len(Attn-LSTM),VideoID,IsPredUseful
0,bczCfVC6HYU,train,"hi , my name is makoto, gramma, i'ma ownership...",0,NaN,NaN,NaN,"[0.077356286346912, 0.35245475172997, 0.182081...","[-0.061935432, -0.06791234, 0.04205813, -0.002...",https://www.youtube.com/watch?v=bczCfVC6HYU,"[9251, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9252,...",13,bczCfVC6HYU,0
1,bczCfVC6HYU,train,let 's do the sesame E of rocking some a hundr...,0,NaN,NaN,NaN,"[0.32907795906067, 0.18735644221306, 0.0040114...","[-0.06560406, 0.049547058, 0.03608154, -0.0293...",https://www.youtube.com/watch?v=bczCfVC6HYU,"[9251, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,...",23,bczCfVC6HYU,0
2,bczCfVC6HYU,train,it 's always my baby after created some on nex...,1,cut salmon into loins,cut,salmon,"[0.70933389663696, 0.23010103404522, 0.0545690...","[-0.034714222, -0.009859278, 0.04199836, 0.033...",https://www.youtube.com/watch?v=bczCfVC6HYU,"[9251, 30, 31, 1, 32, 33, 34, 18, 35, 36, 37, ...",20,bczCfVC6HYU,0
3,bczCfVC6HYU,train,"then start skinning scanning of the sum, and s...",1,skin salmon,skin,salmon,"[0.57078665494919, 0.7331730723381, 0.16216623...","[-0.07715241, 0.06470259, 0.017595463, 0.05354...",https://www.youtube.com/watch?v=bczCfVC6HYU,"[9251, 44, 45, 46, 47, 16, 13, 48, 42, 22, 24,...",30,bczCfVC6HYU,0
4,bczCfVC6HYU,train,this makes the sashimi more beautiful and also...,0,NaN,NaN,NaN,"[0.19342935085297, 0.35593175888062, 0.0840133...","[-0.038894903, 0.049249195, 0.026116883, 0.093...",https://www.youtube.com/watch?v=bczCfVC6HYU,"[9251, 62, 63, 13, 64, 65, 66, 42, 67, 68, 43,...",30,bczCfVC6HYU,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14614,ueGKsfUW-PM,test,so many of them will reduce the temperature of...,1,reduce temperature,reduce,temperature,"[0.44972893595695, 0.22678317129612, 0.0947401...","[-0.023036797, -0.03100028, 0.07529397, 0.0707...",https://www.youtube.com/watch?v=ueGKsfUW-PM,"[9251, 50, 1147, 16, 232, 52, 2129, 13, 684, 1...",21,ueGKsfUW-PM,0
14615,ueGKsfUW-PM,test,so will wait about maybe thirty seconds in bet...,1,drop falafel into oil,drop,falafel,"[0.19766616821289, 0.15502722561359, 0.1278460...","[0.06681934, -0.08966788, 0.045638226, 0.01059...",https://www.youtube.com/watch?v=ueGKsfUW-PM,"[9251, 50, 52, 1302, 414, 346, 84, 2641, 155, ...",22,ueGKsfUW-PM,0
14616,ueGKsfUW-PM,test,so good already will fry them on one side for ...,1,flip falafel,flip,falafel,"[0.045227088034153, 0.17482186853886, 0.177652...","[-0.016999211, -0.06978827, 0.016213309, 0.062...",https://www.youtube.com/watch?v=ueGKsfUW-PM,"[9251, 50, 87, 399, 52, 842, 232, 35, 110, 29,...",32,ueGKsfUW-PM,1
14617,ueGKsfUW-PM,test,i will look beautiful on top they are and of c...,0,NaN,NaN,NaN,"[0.1097567230463, 0.35600194334984, 0.14517596...","[-0.027843257, -0.016666643, 0.08602449, -0.02...",https://www.youtube.com/watch?v=ueGKsfUW-PM,"[9251, 152, 52, 807, 66, 35, 462, 415, 103, 42...",20,ueGKsfUW-PM,0


In [27]:
def get_words(sentence, num=5):
    return " ".join(sentence.split(" ")[:num])

In [28]:
master_df['five_words'] = master_df['Sentence'].apply(lambda x: get_words(x, num=5))

In [30]:
val_df['five_words'] = val_df['summary'].apply(lambda x: get_words(x, num=5))

In [ ]:
val_df

In [31]:
generate_metrics_from_master_and_val_df(master_df, val_df, master_key='five_words', val_key='five_words')

Total true positives (predicted words overlap with gold words): 2108
Total predicted words: 7702
Total gold words: 10439
Precision: 0.27, Recall: 0.2, F1: 0.23


(0.27, 0.2, 0.23)